# Event analysis preparation

Notebook to convert Jeffrey's plotbee files /mnt/storage/work/jchan/final_beepose/one_week_analysis_800/analysis_tracks*
into a single dataframe

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

In [3]:
#%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import glob
from datetime import datetime, timedelta
# from ipywidgets import interact
# from datetime import datetime as dt, time, timedelta

In [4]:
import sys
import os
from os.path import join
import re
#import cv2

from tqdm.notebook import tqdm

### Load raw files one by one to generate an event dataset

In [5]:
def get_video_date(filename):
    pattern  = r"C(?P<colony>[0-9][0-9])_(?P<year>[0-9][0-9])(?P<month>[0-9][0-9])(?P<day>[0-9][0-9])(?P<hour>[0-9][0-9])(?P<min>[0-9][0-9])(?P<sec>[0-9][0-9])"
    pattern = re.compile(pattern)
    m = pattern.search(filename)
    date_format = ["year", "month", "day", "hour", "min", "sec"]

    YY,MM,DD,hh,mm,ss=[int(m.group(g)) for g in date_format]
    return datetime(2000+YY,MM,DD,hh,mm,ss)

def one_hot_encoding(df, columns=[]):
    cdf = df.copy()
    if isinstance(columns, list):
        for c in columns:
            cdf = one_hot_encode_column(cdf, c)
    elif isinstance(columns, str):
        cdf = one_hot_encode_column(cdf, columns)
    return cdf
  
def one_hot_encode_column(df, column):
    categories = df[column].dropna().unique()
    for cat in categories:
        df[cat] = (df[column] == cat)
    return df

def load_track_df(filename):
    track_df = pd.read_csv(filename)
    starttime = get_video_date(filename)
    track_df["datetime"] = track_df.apply(lambda x: starttime + timedelta(seconds=x["track_startframe"]/20), axis=1)
    track_df["track_starttime"] = track_df.apply(lambda x: starttime + timedelta(seconds=x["track_startframe"]/20), axis=1)
    track_df["track_endtime"] = track_df.apply(lambda x: starttime + timedelta(seconds=x["track_endframe"]/20), axis=1)
    track_df["pollen"] = track_df["track_pollen_score"] > 0.5
    track_df = one_hot_encoding(track_df, columns=["track_event"])
    return track_df

In [8]:
# List all JSON files to load
files = glob.glob("/mnt/storage/work/jchan/final_beepose/one_week_analysis_800/analysis_tracks*")
files = sorted(files)
files;

In [9]:
track_dfs = list()
for i, file in enumerate(tqdm(files)):
    track_df = load_track_df(file)
    track_df["video_seq"] = i
    track_dfs.append(track_df)
    
full_df = pd.concat(track_dfs).reset_index() # Important must have unique indices!

In [43]:
# Add path to video (custom)
FILE_PATTERN = "/mnt/storage/work/jchan/bees/tag_dataset/videos/C02_%y%m%d%H0000.mp4"
NAME_PATTERN = "C02_%y%m%d%H0000.mp4"
full_df.loc[:, "video_filename"] = full_df.datetime.apply(lambda x: x.strftime(FILE_PATTERN))
full_df.loc[:, "video_name"] = full_df.datetime.apply(lambda x: x.strftime(NAME_PATTERN))
# Warning: video_id is not the id in labelbee database! Just local id for processing the current dataset
# Which is why we store the full video file name along

In [51]:
#full_df=full_df.drop('index', axis=1)
#full_df['video_seq'] = full_df['video_id']
full_df=full_df.drop('file_name', axis=1)
full_df.columns

Index(['track_id', 'track_pollen_score', 'track_shape', 'track_event',
       'track_tagid', 'track_hastag', 'track_startframe', 'track_startx',
       'track_starty', 'track_starta', 'track_endframe', 'track_endx',
       'track_endy', 'track_enda', 'track_length', 'datetime',
       'track_starttime', 'track_endtime', 'pollen', 'entering', 'leaving',
       'entering_leaving', 'walking', 'video_name', 'labelbee_videoid',
       'video_filename', 'video_seq'],
      dtype='object')

In [52]:
full_df.head()

,track_id,track_pollen_score,track_shape,track_event,track_tagid,track_hastag,track_startframe,track_startx,track_starty,track_starta,...,track_endtime,pollen,entering,leaving,entering_leaving,walking,video_name,labelbee_videoid,video_filename,video_seq
0,0,0.008814,inside,None,NaN,False,0,604,192,37.405357,...,2017-06-21 08:00:03.900,False,False,False,False,False,C02_170621080000.mp4,80,/mnt/storage/work/jchan/bees/tag_dataset/video...,0
1,1,0.035165,ramp-inside,None,NaN,False,0,1856,704,25.866357,...,2017-06-21 08:00:10.100,False,False,False,False,False,C02_170621080000.mp4,80,/mnt/storage/work/jchan/bees/tag_dataset/video...,0
2,2,0.068826,outside_inside,entering,NaN,False,0,1960,924,271.332220,...,2017-06-21 08:00:01.550,False,True,False,False,False,C02_170621080000.mp4,80,/mnt/storage/work/jchan/bees/tag_dataset/video...,0
3,3,0.000026,inside,None,NaN,False,0,1632,256,344.875993,...,2017-06-21 08:00:11.500,False,False,False,False,False,C02_170621080000.mp4,80,/mnt/storage/work/jchan/bees/tag_dataset/video...,0
4,4,0.015290,ramp_ramp,None,NaN,False,0,888,312,320.826342,...,2017-06-21 08:00:05.200,False,False,False,False,False,C02_170621080000.mp4,80,/mnt/storage/work/jchan/bees/tag_dataset/video...,0


In [64]:
# Get labelbee database info
labelbee_df = pd.read_csv('/mnt/storage/Gurabo/datasets/gurabo1/labelbee_videolist_gurabo1.csv')

# Transform the dataframe into a mapping series  file_name => labelbee_videoid
labelbee_mapping = labelbee_df.set_index('file_name')['id']

# Apply the mapping
full_df['labelbee_videoid'] = full_df['video_name'].map(labelbee_mapping)

In [66]:
labelbee_mapping

file_name
C01_180102140000.mp4    10859
C01_171120095841.mp4    10860
C01_170824080000.mp4    10861
C02_170622010000.mp4    10862
C01_170815160000.mp4    10863
                        ...  
C01_180514060000.mp4    16360
C01_180406190000.mp4    16361
C02_170504200000.mp4    16362
C01_180515170000.mp4    16363
C02_170727070000.mp4    16364
Name: id, Length: 5506, dtype: int64

In [65]:
# Save full dataset into a single file
full_df.to_parquet('one_week_analysis_800__tracks__with_labelbee_videoid.parquet')

In [55]:
!ls -hl one_week_analysis_800*.parquet

-rw-rw-r-- 1 rmegret rmegret 80M Mar 12 17:36 one_week_analysis_800__tracks.parquet
-rw-rw-r-- 1 rmegret rmegret 76M Mar 20 11:04 one_week_analysis_800__tracks__with_labelbee_videoid.parquet
-rw-rw-r-- 1 rmegret rmegret 80M Mar 18 23:02 one_week_analysis_800__tracks__with_video_names.parquet


### Bulk load

In [56]:
full_df = pd.read_parquet('one_week_analysis_800__tracks__with_labelbee_videoid.parquet')

In [57]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1958893 entries, 0 to 1958892
Data columns (total 27 columns):
 #   Column              Dtype         
---  ------              -----         
 0   track_id            int64         
 1   track_pollen_score  float64       
 2   track_shape         object        
 3   track_event         object        
 4   track_tagid         float64       
 5   track_hastag        bool          
 6   track_startframe    int64         
 7   track_startx        int64         
 8   track_starty        int64         
 9   track_starta        float64       
 10  track_endframe      int64         
 11  track_endx          int64         
 12  track_endy          int64         
 13  track_enda          float64       
 14  track_length        int64         
 15  datetime            datetime64[ns]
 16  track_starttime     datetime64[ns]
 17  track_endtime       datetime64[ns]
 18  pollen              bool          
 19  entering            bool          
 20  le

# Statistics about the dataset

In [38]:
full_df.shape[0]

1958893

In [35]:
id_counts = full_df['track_tagid'].dropna().astype(int).value_counts().rename('counts')
id_counts.index.groupby(id_counts)

{1: [1974, 2481, 3, 1218, 1142, 2122, 1907, 1962, 155, 2230, 440, 2238, 212, 2262, 2286, 2138, 1951, 1930, 380, 1418, 1454, 1514, 1238, 1582, 1739, 232, 1350, 283, 2265, 1802, 1234, 1854, 1866, 1730, 1567, 287, 3006, 787, 831, 859, 2918, 1627, 64, 2938, 2990, 1039, 739, 1139, 1147, 1151, 1327, 1367, 1387, 2237, 2205, 775, 2774, 319, 467, 2398, 359, 1423, 407, 427, 2502, 962, 455, 2538, 707, 200, 2570, 2578, 2594, 2602, 583, 2666, 2738, 371, 2775, 2991, 2947, 2460, 2464, 2508, 2552, 2993, 2592, 2640, 2700, 928, 2716, 2768, 757, 2820, 781, 2884, 857, 873, 881, 2972, ...], 2: [444, 1432, 1425, 71, 60, 989, 921, 1914, 410, 1643, 2969, 2603, 1562, 2930, 2641, 2899, 249, 2248, 1047, 1156], 3: [1420, 2048, 2855, 2759, 2086, 611, 2444, 912], 4: [578, 397], 5: [986, 2258, 1979], 6: [1109], 7: [661], 12: [677], 15: [930], 16: [1651], 18: [1404], 21: [1415], 27: [1786], 41: [2487], 52: [515], 60: [508], 65: [1722, 1652], 67: [197], 87: [1361], 99: [516], 101: [1900], 109: [638], 142: [1237], 169:

In [92]:
# Select tag ids that have 5 events or more
id_counts = full_df['track_tagid'].dropna().astype(int).value_counts().rename('counts')
tids = id_counts[id_counts>=5].index
tids = np.sort(tids).astype(int)
tids

array([ 197,  508,  515,  516,  517,  638,  661,  677,  930,  986, 1109,
       1230, 1237, 1361, 1404, 1407, 1415, 1607, 1609, 1621, 1651, 1652,
       1698, 1722, 1780, 1786, 1797, 1900, 1979, 2258, 2487])